In [2]:
import xarray as xr
import pandas as pd
data_temp = xr.open_mfdataset(r"C:\Users\iarla\OneDrive\Documents\MSc_Project\HadUK_data\12km_Month_Temp\*.nc", parallel=True)
data_tmax = xr.open_mfdataset(r"C:\Users\iarla\OneDrive\Documents\MSc_Project\HadUK_data\12km_Month_tmax\*.nc", parallel=True)
data_tmin = xr.open_mfdataset(r"C:\Users\iarla\OneDrive\Documents\MSc_Project\HadUK_data\12km_Month_tmin\*.nc", parallel=True)
data_rain = xr.open_mfdataset(r"C:\Users\iarla\OneDrive\Documents\MSc_Project\HadUK_data\12km_Month_Rain\*.nc", parallel=True)
data_hurs = xr.open_mfdataset(r"C:\Users\iarla\OneDrive\Documents\MSc_Project\HadUK_data\12km_Month_Humidity\*.nc", parallel=True)
data_sun = xr.open_mfdataset(r"C:\Users\iarla\OneDrive\Documents\MSc_Project\HadUK_data\12km_Month_Sun\*.nc", parallel=True)
data_frost = xr.open_mfdataset(r"C:\Users\iarla\OneDrive\Documents\MSc_Project\HadUK_data\12km_Month_Frost\*.nc", parallel=True)
data_psl = xr.open_mfdataset(r"C:\Users\iarla\OneDrive\Documents\MSc_Project\HadUK_data\12km_Month_psl\*.nc", parallel=True)
data_wind = xr.open_mfdataset(r"C:\Users\iarla\OneDrive\Documents\MSc_Project\HadUK_data\12km_Month_Wind\*.nc", parallel=True)

In [3]:
import numpy as np

rain = np.array(data_rain['rainfall'])
hurs = np.array(data_hurs['hurs'])
temp = np.array(data_temp['tas'])
temp_max = np.array(data_tmax['tasmax'])
temp_min = np.array(data_tmin['tasmin'])
sun = np.array(data_sun['sun'])
frost = np.array(data_frost['groundfrost'])
psl = np.array(data_psl['psl'])
wind = np.array(data_wind['sfcWind'])

min_length = len(wind)
rain = rain[:min_length]
hurs = hurs[:min_length]
temp = temp[:min_length]
temp_max = temp_max[:min_length]
temp_min = temp_min[:min_length]
sun = sun[:min_length]
frost = frost[:min_length]
wind = wind[:min_length]
psl = psl[:min_length]

In [4]:
import torch 

land_mask = ~np.isnan(temp)
land_mask_torch = torch.from_numpy(land_mask.astype(bool)) 

print(temp.shape)
print(land_mask.shape)

(648, 112, 82)
(648, 112, 82)


In [5]:
from skimage import measure

# Connected components labeling
labels = measure.label(land_mask)
biggest_component_label = np.argmax(np.bincount(labels.flat)[1:]) + 1  

# Isolate the biggest component
main_landmass_mask = (labels == biggest_component_label) 

# Get indices for bounding box
nonzero_indices = main_landmass_mask.nonzero()
min_row = np.min(nonzero_indices[1])
max_row = np.max(nonzero_indices[1])
min_col = np.min(nonzero_indices[2])
max_col = np.max(nonzero_indices[2])

In [6]:
rain = rain[:, min_row:max_row + 1, min_col:max_col + 1]
hurs = hurs[:, min_row:max_row + 1, min_col:max_col + 1]
temp = temp[:, min_row:max_row + 1, min_col:max_col + 1]
temp_max = temp_max[:, min_row:max_row + 1, min_col:max_col + 1]
temp_min = temp_min[:, min_row:max_row + 1, min_col:max_col + 1]
sun = sun[:, min_row:max_row + 1, min_col:max_col + 1] 
frost = frost[:, min_row:max_row + 1, min_col:max_col + 1] 
psl = psl[:, min_row:max_row + 1, min_col:max_col + 1] 
wind = wind[:, min_row:max_row + 1, min_col:max_col + 1] 
land_mask = land_mask[:, min_row:max_row + 1, min_col:max_col + 1]

In [7]:
nan_counts = np.count_nonzero(np.isnan(rain), axis=0)  # Counts across time

valid_grid_cell_mask = nan_counts == 0
valid_cell_indices = np.where(valid_grid_cell_mask)

# Your arrays from the previous output:
row_indices = valid_cell_indices[0]
col_indices = valid_cell_indices[1]

unique_rows, row_counts = np.unique(row_indices, return_counts=True)
unique_cols, col_counts = np.unique(col_indices, return_counts=True)


In [8]:
valid_row_indices = valid_cell_indices[0]  # Extract row indices
valid_col_indices = valid_cell_indices[1]  # Extract column indices

valid_rain_data = rain[:, valid_row_indices, valid_col_indices]
valid_temp_data = temp[:, valid_row_indices, valid_col_indices]
valid_wind_data = wind[:, valid_row_indices, valid_col_indices]
valid_hurs_data = hurs[:, valid_row_indices, valid_col_indices]
valid_psl_data = psl[:, valid_row_indices, valid_col_indices]
valid_frost_data = frost[:, valid_row_indices, valid_col_indices]
valid_sun_data = sun[:, valid_row_indices, valid_col_indices]
valid_tmax_data = temp_max[:, valid_row_indices, valid_col_indices]
valid_tmin_data = temp_min[:, valid_row_indices, valid_col_indices]

In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# List of variables to normalize
variables = [
    valid_rain_data, valid_temp_data, valid_wind_data, 
    valid_hurs_data, valid_psl_data, valid_sun_data,
    valid_frost_data, valid_tmax_data, valid_tmin_data
] 

# Normalize each variable in-place
for data in variables:
    # Fit on assumed training data (replace with your actual training set)
    scaler.fit(data)  

    # Transform and overwrite the existing variable
    data[:] = scaler.transform(data) 

In [10]:
valid_rain_data = valid_rain_data[:,:1600].reshape(648, 64, 25)
valid_temp_data = valid_temp_data[:,:1600].reshape(648, 64, 25)
valid_wind_data = valid_wind_data[:,:1600].reshape(648, 64, 25)
valid_hurs_data = valid_hurs_data[:,:1600].reshape(648, 64, 25)
valid_psl_data = valid_psl_data[:,:1600].reshape(648, 64, 25)
valid_sun_data = valid_sun_data[:,:1600].reshape(648, 64, 25)
valid_frost_data = valid_frost_data[:,:1600].reshape(648, 64, 25)
valid_tmax_data = valid_tmax_data[:,:1600].reshape(648, 64, 25)
valid_tmin_data = valid_tmin_data[:,:1600].reshape(648, 64, 25)

In [11]:
nan_count_temp = np.isnan(valid_temp_data).sum()
nan_count_tmax = np.isnan(valid_tmax_data).sum()
nan_count_tmin = np.isnan(valid_tmin_data).sum()
nan_count_wind = np.isnan(valid_wind_data).sum()

print('Number of NaN values in Temp:', nan_count_temp)
print('Number of NaN values in Tmax:', nan_count_tmax)
print('Number of NaN values in Tmin:', nan_count_tmin)
print('Number of NaN values in Wind:', nan_count_wind)


def fill_nan_with_mean(data):
    col_mean = np.nanmean(data, axis=0)  # Calculate mean per column
    inds = np.where(np.isnan(data))
    data[inds] = np.take(col_mean, inds[1])  # Replace NaNs with mean
    return data

temp_filled = fill_nan_with_mean(valid_temp_data.copy())  
tmax_filled = fill_nan_with_mean(valid_tmax_data.copy())
tmin_filled = fill_nan_with_mean(valid_tmin_data.copy())
wind_filled = fill_nan_with_mean(valid_wind_data.copy())

nan_count_temp = np.isnan(temp_filled).sum()
nan_count_tmax = np.isnan(tmax_filled).sum()
nan_count_tmin = np.isnan(tmin_filled).sum()
nan_count_wind = np.isnan(wind_filled).sum()

print('Number of NaN values in Temp:', nan_count_temp)
print('Number of NaN values in Tmax:', nan_count_tmax)
print('Number of NaN values in Tmin:', nan_count_tmin)
print('Number of NaN values in Tmin:', nan_count_tmin)

Number of NaN values in Temp: 123
Number of NaN values in Tmax: 108
Number of NaN values in Tmin: 123
Number of NaN values in Wind: 167
Number of NaN values in Temp: 0
Number of NaN values in Tmax: 0
Number of NaN values in Tmin: 0
Number of NaN values in Tmin: 0


In [12]:
rain_tensor = torch.from_numpy(valid_rain_data)
temp_tensor = torch.from_numpy(temp_filled)
hurs_tensor = torch.from_numpy(valid_hurs_data)
frost_tensor = torch.from_numpy(valid_frost_data)
sun_tensor = torch.from_numpy(valid_sun_data)
wind_tensor = torch.from_numpy(wind_filled)
psl_tensor = torch.from_numpy(valid_psl_data)
tmax_tensor = torch.from_numpy(tmax_filled)
tmin_tensor = torch.from_numpy(tmin_filled)

In [13]:
variables = [rain_tensor, temp_tensor, hurs_tensor, frost_tensor, sun_tensor, wind_tensor, psl_tensor, tmax_tensor, tmin_tensor] 

# Stack along the channels dimension (dim=1)
stacked_tensor = torch.stack(variables, dim=1) 

stacked_array = stacked_tensor.numpy() 
print('Stacked array shape:', stacked_array.shape)

Stacked array shape: (648, 9, 64, 25)


In [14]:
from convlstm import ConvLSTM
import torch  

seq_length = 12
num_layers = 3  
hidden_dim = 64 
kernel_size = (5, 5)  
input_channels = 9   

model = ConvLSTM(input_dim=input_channels,
                    hidden_dim=hidden_dim,
                    kernel_size=kernel_size,
                    num_layers=num_layers,
                    batch_first=True,  
                    return_all_layers=False)

In [15]:
temperature_data = stacked_array[:, 1, :, :]

print('Temp shape:', temperature_data.shape)

Temp shape: (648, 64, 25)


In [16]:
def prepare_data(data, forecast_horizon):
    xs = []
    ys = []
    max_length = len(data) - forecast_horizon
    for i in range(max_length):
        x = data[i:i + forecast_horizon].copy()  # Take the current forecast horizon
        y = data[i + 1:i + forecast_horizon + 1].copy()  # Take the next forecast horizon
        xs.append(x)
        ys.append(y)

    X = np.array(xs)
    y = np.array(ys)

    return X, y

# Example usage:
forecast_horizon = 12
X, y = prepare_data(stacked_array, forecast_horizon)
print("Shape of prepared input sequences:", X.shape)
print("Shape of prepared target sequences:", y.shape)

Shape of prepared input sequences: (636, 12, 9, 64, 25)
Shape of prepared target sequences: (636, 12, 9, 64, 25)


In [17]:
from sklearn.model_selection import TimeSeriesSplit

def split_data(climate_data, seq_length, forecast_horizon, num_splits=5, test_size=0.2):
    """Splits climate data using Time Series Cross-Validation.

    Args:
        climate_data (np.ndarray): The full climate dataset.
        seq_length (int): Length of input sequences.
        forecast_horizon (int): Length of the forecast horizon.
        num_splits (int): Number of splits for cross-validation.
        test_size (float): Proportion of data to be used as a held-out test set.

    Returns:
        list: A list of tuples, each containing:
              - train_data (np.ndarray): Training data batch.
              - val_data (np.ndarray): Validation data batch.
              - test_data (np.ndarray): Test data batch.
    """
    tscv = TimeSeriesSplit(n_splits=num_splits)
    splits = []

    for train_index, val_index in tscv.split(climate_data):
        X_train, X_val = climate_data[train_index], climate_data[val_index]

        # Hold out a test set from the end
        test_split_index = int(len(X_train) * (1 - test_size)) 
        X_test = X_train[test_split_index:]
        X_train = X_train[:test_split_index]

        train_data = X_train[:-forecast_horizon]  # Adjust for sequence lengths
        val_data = X_val[:-forecast_horizon]
        test_data = X_test[:-forecast_horizon]

        splits.append((train_data, val_data, test_data))

    return splits


In [24]:
from torch.utils.data import Dataset, DataLoader

class ClimateDataset(Dataset):
    def __init__(self, data, targets, seq_length, forecast_horizon): 
        self.data = data  
        self.targets = targets 
        self.seq_length = seq_length
        self.forecast_horizon = forecast_horizon

    def __len__(self):
        return len(self.data) - self.seq_length - self.forecast_horizon + 1

    def __getitem__(self, index):
        X = torch.tensor(self.data[index:index + self.seq_length]).float()  # Shape: (seq_length, channels, height, width)
        y = torch.tensor(self.targets[index + self.seq_length:index + self.seq_length + self.forecast_horizon]).float()  # Shape: (forecast_horizon, channels, height, width)

        print("Shape of a single input sequence (X) from Dataset:", X.shape)
        print("Shape of a single target sequence (y) from Dataset:", y.shape)

        return X, y

def collate_fn(batch):
    # Extract input and target sequences from the batch
    input_sequences = [item[0] for item in batch]  
    target_sequences = [item[1] for item in batch]  
    
    # Concatenate input and target sequences along the batch dimension
    input_tensor = torch.cat(input_sequences, dim=0)
    target_tensor = torch.cat(target_sequences, dim=0)
    
    # Print the shapes of the concatenated tensors
    print('Shape of input tensor:', input_tensor.shape)
    print('Shape of target tensor:', target_tensor.shape)
    
    return input_tensor, target_tensor
    
def create_data_loaders(splits, batch_size=12):
    dataloaders = []
    for train_data, val_data, test_data in splits:
        train_data, train_targets = prepare_data(train_data, forecast_horizon) # Remove seq_length
        val_data, val_targets = prepare_data(val_data, forecast_horizon)
        test_data, test_targets = prepare_data(test_data, forecast_horizon)

        train_dataset = ClimateDataset(train_data, train_targets, seq_length, forecast_horizon)
        val_dataset = ClimateDataset(val_data, val_targets, seq_length, forecast_horizon)
        test_dataset = ClimateDataset(test_data, test_targets, seq_length, forecast_horizon)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)   
        val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)

        dataloaders.append((train_loader, val_loader, test_loader))
    return dataloaders

In [25]:
def train_convlstm(seq_length, forecast_horizon, num_splits, batch_size, num_epochs):
    splits = split_data(stacked_array, seq_length, forecast_horizon, num_splits, 0.2)
    dataloaders = create_data_loaders(splits, batch_size)

    # Initialize your ConvLSTM model
    model = ConvLSTM(input_dim=input_channels,
                     hidden_dim=hidden_dim,
                     kernel_size=kernel_size,
                     num_layers=num_layers,
                     batch_first=True,
                     return_all_layers=False)

    # Loss function and Optimizer
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for train_loader, val_loader, _ in dataloaders:
            for input_seqs, target_seqs in train_loader:
                
                
                optimizer.zero_grad()
                outputs = model(input_seqs)
                print("Shape of model output before loss calculation:", outputs.shape)
                print("Shape of target:", target_seqs.shape)
                loss = criterion(outputs, target_seqs)
                train_loss += loss.item()
                loss.backward()
                optimizer.step()

        # Validation
        model.eval()
        val_losses = []
        r2_scores = []

        with torch.no_grad():
            for input_seqs, target_seqs in val_loader:

                outputs = model(input_seqs)
                val_loss = criterion(outputs, target_seqs)
                val_losses.append(val_loss.item())

                # R-squared calculation
                r2 = r2_score(target_seqs.detach().numpy(), outputs.detach().numpy())
                r2_scores.append(r2)

        mean_val_loss = np.mean(val_losses)
        mean_r2 = np.mean(r2_scores)
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {mean_val_loss:.4f}, Val R^2: {mean_r2:.4f}')

    # Plotting
    plt.figure(figsize=(10, 5))
    plt.plot(training_losses, label='Training Loss')
    plt.plot(validation_losses, label='Validation Loss')
    plt.plot(validation_r2, label='Validation R^2')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (MSE) / R^2')
    plt.legend()
    plt.title('Training and Validation Performance')
    plt.show()
    

In [ ]:
seq_length = 12
forecast_horizon = 12
num_splits = 5
batch_size = 12
num_epochs = 10

train_convlstm(seq_length, forecast_horizon, num_splits, batch_size, num_epochs)

Shape of a single input sequence (X) from Dataset: torch.Size([12, 12, 9, 64, 25])
Shape of a single target sequence (y) from Dataset: torch.Size([12, 12, 9, 64, 25])
Shape of a single input sequence (X) from Dataset: torch.Size([12, 12, 9, 64, 25])
Shape of a single target sequence (y) from Dataset: torch.Size([12, 12, 9, 64, 25])
Shape of a single input sequence (X) from Dataset: torch.Size([12, 12, 9, 64, 25])
Shape of a single target sequence (y) from Dataset: torch.Size([12, 12, 9, 64, 25])
Shape of a single input sequence (X) from Dataset: torch.Size([12, 12, 9, 64, 25])
Shape of a single target sequence (y) from Dataset: torch.Size([12, 12, 9, 64, 25])
Shape of a single input sequence (X) from Dataset: torch.Size([12, 12, 9, 64, 25])
Shape of a single target sequence (y) from Dataset: torch.Size([12, 12, 9, 64, 25])
Shape of a single input sequence (X) from Dataset: torch.Size([12, 12, 9, 64, 25])
Shape of a single target sequence (y) from Dataset: torch.Size([12, 12, 9, 64, 25]